In [4]:
from elasticsearch import helpers, Elasticsearch
import csv
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from inflection import singularize
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine as cs

In [5]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.info()

{'name': 'm-c02wv1jnhtd5',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'i0Q_KCKeTLGlRZzF8qt__A',
 'version': {'number': '7.6.2',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'ef48eb35cf30adf4db14086e8aabd07ef6fb113f',
  'build_date': '2020-03-26T06:34:37.794943Z',
  'build_snapshot': False,
  'lucene_version': '8.4.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [6]:
def text_preprocessing(text):
    text= text.lower()
    text= re.sub(r'[^a-z]',' ',text)
    #text= " ".join([s for s in text.split() if len(s)>2])
    #text= " ".join([x for x in text.split() if x not in stopwords.words('english')])
    #text= " ".join([inflection.singularize(x) for x in text.split()])
    text= text.strip()
    return(text)

In [87]:
index='index_v1'

In [88]:
def doc_generator(data,index):
    df_iter= data.iterrows()
    
    for index,document in df_iter:
        yield{
            "_index": 'index_v1',
            "_type": "_doc",
            "_id" : f"{index}",
            "_source": document.to_json()
        }
    raise StopIteration

In [89]:
for success,info in tqdm(helpers.parallel_bulk(es,doc_generator(df_ques[['pid','passage']],index))):
    if not success:
        print('A document failed:', info)
es.indices.refresh(index=index)

20000it [00:03, 5580.47it/s]


{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}

In [90]:
def SEARCH(text,index,field):
    
    res= es.search(index=index,body={"query":{"match":{field:{"query":text,"operator":"or","fuzziness": "0"
                                                            }}}},size = 50)
    
    return([(x.get('_source'),x.get('_score')) for x in res['hits']['hits']])

In [91]:
index='index_v1'
field = 'passage'

In [93]:
def query_generator(df):
    while True:
        yield (df_new['question'])

In [94]:
query_generator(df_new[['question']])

<generator object query_generator at 0x10f24c200>